# Exercicio 6 - Amostragem e Discretização

Dupla: Giovani Cancherini e Eduardo Traunig

---

**Contexto:**

O processo de amostragem consiste em selecionar um subconjunto dos dados para análise, motivado por vezes pelos requerimentos de processamento e memória. É crucial que as amostras sejam representativas, mantendo propriedades semelhantes às do conjunto de dados original. Existem diferentes tipos de amostragem:

* **Amostragem Aleatória Simples:** Com ou sem reposição.
* **Amostragem Estratificada:** Mantém a proporção dos estratos (categorias), sendo proporcional ou desproporcional (útil em datasets desbalanceados).
* **Amostragem por Conglomerado:** Mantém a proporção dos grupos.

A **Discretização** de atributos contínuos envolve decidir o número de categorias e uma função de mapeamento para converter valores contínuos em categorias, podendo ser representada por intervalos.

Para atributos categóricos com muitas variações, pode ser necessário diminuir o número de categorias. Para variáveis ordinais, pode-se aplicar discretização; para nominais, conhecimento de domínio é essencial.

A **Binarização e One-Hot Encoding** são técnicas de codificação de valores categóricos, transformando-os em representações numéricas binárias, úteis para algoritmos de Machine Learning.

## 1. Importação de Bibliotecas e Carregamento do Dataset

Primeiramente, importaremos as bibliotecas necessárias para as operações de manipulação de dados e amostragem, e carregaremos o dataset `alugueis.csv`.

In [ ]:
import pandas as pd
import numpy as np

# Carregar o dataset 'alugueis.csv'
try:
    df_alugueis = pd.read_csv('alugueis.csv')
    print("Dataset 'alugueis.csv' carregado com sucesso!")
    print("Primeiras 5 linhas do DataFrame:")
    print(df_alugueis.head())
    print("\nInformações do DataFrame:")
    df_alugueis.info()
except FileNotFoundError:
    print("Erro: O arquivo 'alugueis.csv' não foi encontrado. Certifique-se de que está no mesmo diretório.")
   

## 2. Amostragem Aleatória

**Pergunta:** Escreva o corpo da função `amostragem_aleatória(dataframe, amostras, reposicao=True)`.

Essa função recebe três parâmetros:
* `dataframe`: pandas DataFrame contendo os dados.
* `amostras`: número de amostras desejadas.
* `reposicao`: amostragem com ou sem reposição.

Essa função deve retornar um DataFrame com o número exato de amostras. **Não pode usar as funções de amostragem prontas do pandas!**

### Implementação da Função `amostragem_aleatoria`

In [ ]:
def amostragem_aleatoria(dataframe, amostras, reposicao=True):
    num_linhas = len(dataframe)
    indices_selecionados = []

    if reposicao:
        for _ in range(amostras):
            idx = np.random.randint(0, num_linhas)
            indices_selecionados.append(idx)
    else:
        if amostras > num_linhas:
            # Se tentar pegar mais amostras do que o disponível sem reposição,
            # retorne todas as linhas disponíveis. É o máximo que pode ser feito.
            return dataframe.reset_index(drop=True)
        
        indices_disponiveis = list(range(num_linhas))
        for _ in range(amostras):
            idx_selecionado = np.random.randint(0, len(indices_disponiveis))
            indices_selecionados.append(indices_disponiveis.pop(idx_selecionado)) # Remove o index selecionado

    return dataframe.iloc[indices_selecionados].reset_index(drop=True)

### Teste da Função `amostragem_aleatoria`

Vamos criar um dataset com 100 amostras usando a função desenvolvida.

In [ ]:
print("--- Teste de Amostragem Aleatória (100 amostras com reposição) ---")
df_amostra_100_com_reposicao = amostragem_aleatoria(df_alugueis, 100, reposicao=True)
print(f"Shape do dataset amostrado: {df_amostra_100_com_reposicao.shape}")
print("Primeiras 5 linhas da amostra:")
print(df_amostra_100_com_reposicao.head())

print("\n--- Teste de Amostragem Aleatória (100 amostras sem reposição) ---")
if len(df_alugueis) >= 100:
    df_amostra_100_sem_reposicao = amostragem_aleatoria(df_alugueis, 100, reposicao=False)
    print(f"Shape do dataset amostrado: {df_amostra_100_sem_reposicao.shape}")
    print("Primeiras 5 linhas da amostra:")
    print(df_amostra_100_sem_reposicao.head())
else:
    print("O DataFrame original tem menos de 100 linhas. Não é possível realizar amostragem sem reposição para 100 amostras.")

## 3. Amostragem Estratificada

**Pergunta:** Escreva uma função com a assinatura mostrada abaixo: `amostragem_estratificada(dataframe, amostras, coluna)`.

Essa função recebe três parâmetros:
* `dataframe`: pandas DataFrame contendo os dados.
* `amostras`: número de amostras desejadas.
* `coluna`: nome de uma coluna do dataset (para estratificação).

Essa função deve retornar um DataFrame com o número exato de amostras, mantendo a proporção de valores da coluna escolhida presente no DataFrame original.

**Exemplo:**
Dataset Entrada (100 registros)
| Valor da Coluna | Qtde Valores |
| :-------------- | :----------- |
| A               | 50           |
| B               | 30           |
| C               | 10           |
| D               | 20           |

Dataset Saída (10 registros)
| Valor da Coluna | Qtde Valores |
| :-------------- | :----------- |
| A               | 5            |
| B               | 3            |
| C               | 1            |
| D               | 2            |

### Implementação da Função `amostragem_estratificada` corrigida apos extouro de index

In [ ]:
def amostragem_estratificada(dataframe, amostras, coluna):
    if coluna not in dataframe.columns:
        raise ValueError(f"A coluna '{coluna}' não existe no DataFrame.")

    proporcoes = dataframe[coluna].value_counts(normalize=True)
    df_amostrado = pd.DataFrame()

    # Calcular o número de amostras para cada estrato usando floor
    # para garantir que a soma inicial não exceda 'amostras'.
    n_amostras_por_estrato = (proporcoes * amostras).apply(np.floor).astype(int)
    
    # Ajustar a soma para ser exatamente 'amostras' distribuindo a diferença
    # para os estratos com as maiores frações decimais.
    total_amostras_calculadas = n_amostras_por_estrato.sum()
    diferenca = amostras - total_amostras_calculadas

    # Distribui a diferença para os estratos com as maiores frações decimais (ajuste fino)
    fracoes = (proporcoes * amostras) - n_amostras_por_estrato
    indices_para_adicionar = fracoes.nlargest(diferenca).index
    n_amostras_por_estrato[indices_para_adicionar] += 1

    # Iterar sobre cada estrato para coletar as amostras
    for estrato, n_amostras_target in n_amostras_por_estrato.items():
        df_estrato = dataframe[dataframe[coluna] == estrato]

        # Garante que não se tente amostrar mais linhas do que o estrato possui
        n_amostras_real = min(n_amostras_target, len(df_estrato))
        
        if n_amostras_real > 0:
            # Utiliza a função de amostragem aleatória implementada anteriormente (sem reposição)
            amostra_estrato = amostragem_aleatoria(df_estrato, n_amostras_real, reposicao=False)
            df_amostrado = pd.concat([df_amostrado, amostra_estrato])
            
    # Verificação final e ajuste para o número exato de amostras
    # Se o número de amostras coletadas for menor que o desejado, 
    # retorna o que foi possível coletar (já embaralhado)
    if len(df_amostrado) < amostras:
        print(f"Aviso: Não foi possível obter o número exato de {amostras} amostras mantendo as proporções e sem reposição. Retornando {len(df_amostrado)} amostras.")
        return df_amostrado.sample(frac=1, random_state=42).reset_index(drop=True) # Apenas embaralha
    else:
        # Caso contrário, amostra o número exato de forma aleatória e sem reposição
        return df_amostrado.sample(n=amostras, random_state=42, replace=False).reset_index(drop=True)


### Teste da Função `amostragem_estratificada`

Vamos criar um dataset com 200 amostras, preservando a proporção de imóveis em cada cidade.

In [ ]:
print("--- Teste de Amostragem Estratificada (200 amostras por 'city') ---")
if 'city' in df_alugueis.columns:
    print("\nProporção original de Cidades no DataFrame completo:")
    proporcao_original = df_alugueis['city'].value_counts(normalize=True).sort_index()
    print(proporcao_original)

    try:
        df_amostra_200_estratificada_cidade = amostragem_estratificada(df_alugueis, 200, 'city')
        print(f"\nShape do dataset amostrado: {df_amostra_200_estratificada_cidade.shape}")

        print("\nProporção de Cidades na amostra estratificada:")
        proporcao_amostra = df_amostra_200_estratificada_cidade['city'].value_counts(normalize=True).sort_index()
        print(proporcao_amostra)

        print("\nComparação das proporções (Original vs. Amostra):")
        comparacao_proporcoes = pd.DataFrame({
            'Proporção Original': proporcao_original,
            'Proporção na Amostra': proporcao_amostra
        }).fillna(0) 
        print(comparacao_proporcoes)
        
        print("\nPrimeiras 5 linhas da amostra estratificada:")
        print(df_amostra_200_estratificada_cidade.head())

    except ValueError as e:
        print(f"Erro ao realizar amostragem estratificada: {e}")
else:
    print("Coluna 'city' não encontrada no DataFrame. Impossível realizar amostragem estratificada por cidade.")

## 4. Discretização do Atributo 'area' do Imóvel

**Pergunta:** Faça a discretização do atributo `area` do imóvel em três categorias:
* "PEQUENO" = até 50m²
* "MÉDIO" = 50 até 100m²
* "GRANDE" = > 100m²

In [ ]:
print("--- Discretização do atributo 'area' ---")
if 'area' in df_alugueis.columns:
    # Definir os limites dos intervalos e os rótulos das categorias
    # [0, 50) -> PEQUENO
    # [50, 100) -> MÉDIO
    # [100, inf) -> GRANDE
    bins = [0, 50, 100, np.inf]
    labels = ['PEQUENO', 'MÉDIO', 'GRANDE']
    
    # Realizar a discretização usando pd.cut
    df_alugueis['area_categoria'] = pd.cut(df_alugueis['area'], bins=bins, labels=labels, right=False)

    print("\nContagem de valores para a nova coluna 'area_categoria':")
    print(df_alugueis['area_categoria'].value_counts().sort_index())
    print("\nExemplo de linhas com as colunas 'area' e 'area_categoria':")
    print(df_alugueis[['area', 'area_categoria']].head(10))
else:
    print("Coluna 'area' não encontrada no DataFrame. Impossível realizar a discretização.")

## 5. Discretização e One-Hot Encoding do Atributo 'total'

**Pergunta:** Divida o atributo `total` em 5 categorias e utilize One-Hot Encoding para discretizá-lo.

In [ ]:
print("--- Discretização e One-Hot Encoding do atributo 'total' ---")
if 'total' in df_alugueis.columns:
    num_categorias_total = 5
    
    # Discretização do atributo 'total' em 5 categorias de igual frequência (quantis)
    # labels=False retorna inteiros para as categorias (0, 1, 2, 3, 4)
    # duplicates='drop' lida com valores idênticos que poderiam criar bins vazios
    df_alugueis['total_categoria'] = pd.qcut(df_alugueis['total'], q=num_categorias_total, labels=False, duplicates='drop')

    print(f"\nContagem de valores para a nova coluna 'total_categoria' ({num_categorias_total} categorias):")
    print(df_alugueis['total_categoria'].value_counts().sort_index())
    print("\nExemplo de linhas com as colunas 'total' e 'total_categoria':")
    print(df_alugueis[['total', 'total_categoria']].head(10))

    # Aplicação de One-Hot Encoding nas categorias do 'total'
    df_total_one_hot = pd.get_dummies(df_alugueis['total_categoria'], prefix='total_cat')

    print("\nDataFrame resultante do One-Hot Encoding para 'total_categoria':")
    print(df_total_one_hot.head())

    # Opcional: Adicionar as colunas de One-Hot Encoding ao DataFrame original
    # Isso é comum para integrar as novas features ao conjunto de dados principal
    df_alugueis = pd.concat([df_alugueis, df_total_one_hot], axis=1)
    print("\nPrimeiras 5 linhas do DataFrame 'alugueis' após adicionar as colunas de One-Hot Encoding:")
    print(df_alugueis.head())
    print(f"\nShape final do DataFrame após One-Hot Encoding: {df_alugueis.shape}")
else:
    print("Coluna 'total' não encontrada no DataFrame. Impossível realizar discretização e One-Hot Encoding.")